In [3]:
#Importing libraries
import requests
import pandas as pd
import datetime
import time

#using openweather api
API_KEY = 'a4c2cf6491d1f1546ad9660eb68e87ea'  
BASE_URL = 'https://history.openweathermap.org/data/3.0/history/timemachine'

#adding states cordinations alphabitically sorted 
state_coordinates = {
    "Alabama": (32.377716, -86.300568),
    "Alaska": (58.301598, -134.420212),
    "Arizona": (33.448376, -112.074036),
    "Arkansas": (34.746613, -92.288986),
    "California": (38.576668, -121.493629),
    "Colorado": (39.739235, -104.990250),
    "Connecticut": (41.765804, -72.673372),
    "Delaware": (39.157307, -75.519722),
    "Florida": (30.438118, -84.281296),
    "Georgia": (33.749027, -84.388229),
    "Hawaii": (21.307442, -157.857376),
    "Idaho": (43.617775, -116.199722),
    "Illinois": (39.798363, -89.654961),
    "Indiana": (39.768623, -86.162643),
    "Iowa": (41.591087, -93.603729),
    "Kansas": (39.048191, -95.677956),
    "Kentucky": (38.186722, -84.875374),
    "Louisiana": (30.457069, -91.187393),
    "Maine": (44.307167, -69.781693),
    "Maryland": (38.978764, -76.490936),
    "Massachusetts": (42.360253, -71.058291),
    "Michigan": (42.733635, -84.555328),
    "Minnesota": (44.955097, -93.102211),
    "Mississippi": (32.298757, -90.184810),
    "Missouri": (38.576702, -92.173516),
    "Montana": (46.585709, -112.018417),
    "Nebraska": (40.808075, -96.699654),
    "Nevada": (39.163914, -119.766121),
    "New Hampshire": (43.206898, -71.537994),
    "New Jersey": (40.220596, -74.769913),
    "New Mexico": (35.68224, -105.939728),
    "New York": (42.652843, -73.757874),
    "North Carolina": (35.78043, -78.639099),
    "North Dakota": (46.82085, -100.783318),
    "Ohio": (39.961346, -82.999069),
    "Oklahoma": (35.492207, -97.503342),
    "Oregon": (44.938461, -123.030403),
    "Pennsylvania": (40.264378, -76.883598),
    "Rhode Island": (41.830914, -71.414963),
    "South Carolina": (34.000343, -81.033211),
    "South Dakota": (44.367031, -100.346405),
    "Tennessee": (36.162663, -86.781601),
    "Texas": (30.274670, -97.740349),
    "Utah": (40.777477, -111.888237),
    "Vermont": (44.262436, -72.580536),
    "Virginia": (37.538857, -77.433640),
    "Washington": (47.035805, -122.905014),
    "West Virginia": (38.336246, -81.612328),
    "Wisconsin": (43.074684, -89.384445),
    "Wyoming": (41.140259, -104.820236)
}

#getting the weathe data
def get_weather_data(lat, lon, month, year):
    days_in_month = (datetime.date(year, month % 12 + 1, 1) - datetime.timedelta(days=1)).day
    weather_data = []

    for day in range(1, days_in_month + 1):
        date = datetime.datetime(year, month, day)
        timestamp = int(date.timestamp())
        params = {
            'lat': lat,
            'lon': lon,
            'dt': timestamp,
            'appid': API_KEY,
            'units': 'metric'
        }
        
        try:
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()  # Raise HTTPError for bad responses
            daily_data = response.json().get('current', {})
            weather_data.append(daily_data)
        except requests.exceptions.HTTPError as errh:
            print(f"HTTP Error: {errh} - for coordinates: {lat}, {lon} on {date}")
            break
        except requests.exceptions.ConnectionError as errc:
            print(f"Error Connecting: {errc} - for coordinates: {lat}, {lon} on {date}")
            break
        except requests.exceptions.Timeout as errt:
            print(f"Timeout Error: {errt} - for coordinates: {lat}, {lon} on {date}")
            break
        except requests.exceptions.RequestException as err:
            print(f"Oops: Something Else: {err} - for coordinates: {lat}, {lon} on {date}")
            break

        # To avoid hitting rate limits
        time.sleep(1)  # Delay to prevent exceeding rate limits

    return weather_data

def aggregate_monthly_data(state_name, month, year, weather_data):
    total_temp = 0
    sunny_days = 0
    rainy_days = 0
    cloudy_days = 0
    count = len(weather_data)

    for day in weather_data:
        total_temp += day.get('temp', 0)
        weather_condition = day.get('weather', [{}])[0].get('main', '').lower()
        if weather_condition == 'clear':
            sunny_days += 1
        elif weather_condition in ['rain', 'drizzle']:
            rainy_days += 1
        else:
            cloudy_days += 1

    average_temp = total_temp / count if count else 0
    return {
        "State": state_name,
        "Year": year,
        "Month": month,
        "Average Temperature": average_temp,
        "Sunny Days": sunny_days,
        "Rainy Days": rainy_days,
        "Cloudy Days": cloudy_days
    }

def fetch_and_save_weather_data(states, year):
    all_data = []

    for state, (lat, lon) in states.items():
        print(f"Fetching data for {state}...")
        for month in range(1, 13):
            weather_data = get_weather_data(lat, lon, month, year)
            if weather_data:
                monthly_data = aggregate_monthly_data(state, month, year, weather_data)
                all_data.append(monthly_data)

    df = pd.DataFrame(all_data)
    df.to_csv('weather_data.csv', index=False)
    print("Weather data saved to weather_data.csv")

# Fetch and save weather data
fetch_and_save_weather_data(state_coordinates, 2023)

Fetching data for Alabama...
HTTP Error: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/history/city?lat=%7Blat%7D&lon=%7Blon%7D&type=hour&start=%7Bstart%7D&end=%7Bend%7D&appid=%7BAPI%20key%7D&lat=32.377716&lon=-86.300568&dt=1672527600&appid=a4c2cf6491d1f1546ad9660eb68e87ea&units=metric - for coordinates: 32.377716, -86.300568 on 2023-01-01 00:00:00
HTTP Error: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/history/city?lat=%7Blat%7D&lon=%7Blon%7D&type=hour&start=%7Bstart%7D&end=%7Bend%7D&appid=%7BAPI%20key%7D&lat=32.377716&lon=-86.300568&dt=1675206000&appid=a4c2cf6491d1f1546ad9660eb68e87ea&units=metric - for coordinates: 32.377716, -86.300568 on 2023-02-01 00:00:00
HTTP Error: 401 Client Error: Unauthorized for url: https://history.openweathermap.org/data/2.5/history/city?lat=%7Blat%7D&lon=%7Blon%7D&type=hour&start=%7Bstart%7D&end=%7Bend%7D&appid=%7BAPI%20key%7D&lat=32.377716&lon=-86.300568&dt=1677625200&appid=a

KeyboardInterrupt: 